In [2]:
import pandas as pd
import re

from pathlib import Path
from tqdm.notebook import tqdm

In [5]:
fs = lambda x: str(x)+'1' if re.findall(r"(-S)\b", str(x)) else str(x)  # Add S1
fc = lambda x: str(x)+'1' if re.findall(r"(-C)\b", str(x)) else str(x)  # Add C1


labelPaths = sorted(Path.cwd().parent.joinpath('data', 'Label').glob('*.txt')) + \
  sorted(Path.cwd().parent.joinpath('data', 'OpenSource').glob('*.txt'))

for labelPath in tqdm(labelPaths):
  df = pd.read_csv(labelPath, sep='\t', names=['start_time', 'end_time', 'species'])
  df = df[df['start_time'] != '\\']
  df['species'] = df['species'].apply(lambda x: ''.join(str(x).split()))
  df['species'] = df['species'].apply(fs)
  df['species'] = df['species'].apply(fc)
  df.to_csv(labelPath, sep='\t', header=False, index=False, encoding='utf-8')

  0%|          | 0/1741 [00:00<?, ?it/s]

In [6]:
# FileSpeciesList file
import warnings
warnings.filterwarnings('ignore')

labelPaths = sorted(Path.cwd().parent.joinpath('data', 'Label').glob('*.txt'))

fileSpeciesListDF = pd.DataFrame(index=[labelPath.name for labelPath in labelPaths])

for labelPath in tqdm(labelPaths):
  df = pd.read_csv(labelPath, sep='\t', names=['start_time', 'end_time', 'species'])
  tempSeries = df['species'].value_counts()
  for sp in df['species'].unique():
    if not sp in fileSpeciesListDF.columns:
      fileSpeciesListDF[sp] = 0
  for i, x in enumerate(tempSeries):
    fileSpeciesListDF.loc[labelPath.name, tempSeries.index[i]] += x

fileSpeciesListDF = fileSpeciesListDF.reindex(sorted(fileSpeciesListDF.columns), axis=1)
fileSpeciesListDF.to_csv(Path.cwd().parent.joinpath('report', 'table', 'fileSpeciesList.csv'), header=True, index=True)

  0%|          | 0/1243 [00:00<?, ?it/s]

In [ ]:
# SongTypeCount file
# Visualise
import collections
import matplotlib.pyplot as plt

countDict = {}

for labelPath in tqdm(labelPaths):
  df = pd.read_csv(labelPath, sep='\t', names=['start_time', 'end_time', 'species'])
  df = df[df['species'].str.contains('-S+', regex=True, na=False)]
  df.reset_index(drop=True, inplace=True)

  if df.empty:
    continue

  for _, x in df.iterrows():
    if not countDict.get(x['species']):
      countDict[x['species']] = 0
    countDict[x['species']] += 1

orderCountDict = collections.OrderedDict(sorted(countDict.items()))
fig = plt.figure(figsize=(16, 9))
plt.bar(range(len(orderCountDict)), list(orderCountDict.values()), tick_label=list(orderCountDict.keys()))
plt.xticks(rotation=45)
plt.show()
countDF = pd.DataFrame.from_dict(orderCountDict, orient='index')
countDF.to_csv(Path.cwd().parent.joinpath('report', 'table', 'songTypeCount.csv'), header=False, index=True)